In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
import os
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser

embeddings = OllamaEmbeddings(model='nomic-embed-text:v1.5')

path_books = os.path.join(os.path.dirname(__file__), "vector-store", "books")
db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)
print(db.index.ntotal)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 3

template = '''
    Você é um assistente de busca de livros.

    Histórico de conversa: {chat_history}

    Pergunta: {question}
    '''
prompt = PromptTemplate(
    input_variables=["chat_history", "question"], template=template
)

llm = Ollama(model="mistral:7b")